# Tinkering notebook 2: Markov Decision Processes and Dynamic Programming
In this notebook we will see how some of the content of Lecture 2 - Lecture 3 works in practice. 

We will start by a repetition of Markov Decision Processes (MDPs) and value functions. 
Then we introduce the two GridWorld environments that will be used in the notebook (Example 4.1 in the textbook and FrozenLake). 

We then use dynamic programming to estimate the value function $v_\pi(s)$ for a given policy $\pi$ and then use policy improvement to find the optimal policy. Finally we try value iteration to directly find the optimal value function $v_*(s)$ and the corresponding optimal policy. 

# Table of content
* ### [1. Imports](#sec1)
* ### [2. Markov Decision Processes and Value Functions](#sec2)
 * #### [2.1 The Bellman Equations](#sec2_1)
 * #### [2.2 Example: Study or Facebook?](#sec2_2)
 * #### [2.3 *Analytical solution to the Bellman equation](#sec2_3)
* ### [3. Helper functions](#sec3)
* ### [4. The environments](#sec4)
 * #### [4.1 The Frozen Lake](#sec4_1)
* ### [5. MDPs and the Bellman equations](#sec5)
 * #### [5.1 Test your code on Example 4.1 (GridWorld)](#sec5_1)
* ### [6. Policy Evaluation](#sec6)
 * #### [6.1 In place updates](#sec6_1)
* ### [7. Policy Iteration](#sec7)
* ### [8. Value iteration](#sec8)


# 1. Imports <a id="sec1">

In [ ]:
# Packages needed for this notebook
import gymnasium as gym
import gym_RLcourse
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output # Used to clear the ouput of a Jupyter cell.

# 2. Markov Decision Processes and Value Functions <a id="sec2">

An MDP consist of a state space $\mathcal{S}$, an action space $\mathcal{A}$, a reward set $\mathcal{R}$ and a transition function $p(s', r | s, a)$. We define the return as 
$$
G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \ldots
$$
where $\gamma$ is the discount rate.

**State-value function:**
$$
v_{\pi}(s) = \mathbb{E}_{\pi}[ G_t | S_t = s]
$$
The expected return starting from state $s \in \mathcal{S}$ and following policy $\pi$.

**Action-value function ($Q$-value)**:
$$
q_{\pi}(s,a) = \mathbb{E}_{\pi}[ G_t | S_t = s, A_t = a]
$$
The expected return starting from state $s \in \mathcal{S}$, then taking action $a \in \mathcal{A}$ and then following policy $\pi$.

## 2.1 The Bellman Equations <a id="sec2_1">

We have seen that $v_{\pi}(s)$ is the solution to the Bellman equation
$$
\begin{aligned}
v_{\pi}(s) &= \mathbb{E}_{\pi}[ R_{t+1} + \gamma v_{\pi}(S_{t+1}) | S_t = s] \\
&= \sum_{a} \pi(a|s) \sum_{r} \sum_{s'} p(s', r | s, a) [ r + \gamma v_\pi(s') ] = \sum_{a} \pi(a|s) q_{\pi}(s,a)
\end{aligned}
$$
where
$$
q_{\pi}(s,a) = \mathbb{E}_{\pi}[ G_t | S_t = s, A_t = a] = \sum_{r}\sum_{s'} p(s',r | s,a) [ r + \gamma v_{\pi}(s')]
$$

## 2.2 Example: Study or Facebook? <a id="sec2_2">

In Lecture 2 we looked at the example MDP bellow.
<img src="example.png">

It has four states, $\mathcal{S} = \{ K_0, K_1, K_2, Pass \}$. The state $Pass$ is a terminal state, so the episode will end if this state is reached (alternatively, if we reach $Pass$ we will stay there forever and receive 0 future return).

In each non-terminal state we can choose between `Study` or `Facebook`, $\mathcal{A} = \{ Study, Facebook\}$. The red nodes corresponds to actions, and the labels on the edges gives immediate rewards (green) and transition probabilities (black).

***
### Task:
Assume that we use the policy $\pi(a|s) = 0.5$ for all states and actions, and the discount factor $\gamma = 0.9$.  

In Lecture 2 we saw that the state-value function (rounded to two decimals) is
$$
v_{\pi}(K_0) = 3.00, \quad v_{\pi}(K_1) = 4.78, \quad v_{\pi}(K_2) = 7.84.
$$
1. Verify that this satisfies the Bellman equation for all states (when rounded to two decimals)! 

You can use the code-block bellow to carry out your computations.
***

In [ ]:
gamma = 0.9
vK0 = 0.5*(0 + gamma*3) + 0.5*(-1+gamma*4.78) 
vK1 = 0.5*(0 + gamma*(0.5*3 + 0.5*4.78)) + 0.5*(-1 + gamma*7.84) 
vK2 = 0.5*(0 + gamma*(0.5*4.78 + 0.5*7.84)) + 0.5*(10 + gamma*0) 
print("v(K0) = %.2f, v(K1) = %.2f, v(K2) = %.2f" % (vK0, vK1, vK2))

## 2.3 *Analytical solution to the Bellman equation <a id="sec2_3">

Note that the Bellman equation can be written as
$$
v_{\pi}(s) = \mathbb{E}_{\pi}[ R_{t+1} | S_{t} = s] + \gamma\sum_{a, s'} \pi(a|s)p(s' | s, a) v_{\pi}(s').
$$
So the value of $s$ is the average immediate reward plus the discounted average value of the next state.

**Example: From the state $K_1$**:
    
* In $s = K_1$ the expected immediate reward is $-0.5$, since we choose `Facebook` with probability 0.5 (reward 0) and `Study` with probability 0.5 (reward -1). 
* There is a 0.5 probability that the action is `Facebook`, and then there is a 50/50 chance of going either to $K_0$ or $K_1$. Hence the total probability of going to $K_0$ and $K_1$ are both 0.25 ($0.5 \times 0.5$). There is also a 0.5 probability for the action `Study` which will move us to $K_2$. Finally there is 0 probability of reaching $Pass$. 

Summarizing this we get
$$
v_{\pi}(K_1) = -0.5 + \gamma [ 0.25 v_{\pi}(K_0) + 0.25 v_{\pi}(K_1) + 0.5 v_{\pi}(K_2) + 0 v_{\pi}(Pass)]
$$
If we define a vector with all state-values
$$
V_{\pi} = \begin{bmatrix} v_{\pi}(K_0) \\ v_{\pi}(K_1) \\ v_{\pi}(K_2) \\ v_{\pi}(Pass) \end{bmatrix}
$$
we can write this as
$$
v_{\pi}(K_1) = \underbrace{-0.5}_{r_1} + \gamma \underbrace{\begin{bmatrix} 0.25 & 0.25 & 0.5 & 0 \end{bmatrix}}_{p_1} V_\pi
$$
Note that the elements of $p_1$ are the probability of going from $K_1$ to each of the other states when we follow the policy.

**Putting it all together:**

If we do the same for all states, we get an equation on the form 
$$
V_{\pi} = R + \gamma P V_{\pi} \iff (I - \gamma P) V_{\pi} = R
$$
where $R$ is the vector of expected immediate rewards for each state, and $P \in \mathbb{R}^{4 \times 4}$ where element $P_{i,j}$ is the probability of moving from the $i$th state to the $j$th state when we follow the policy.

Assuming that $\gamma < 1$ then $I - \gamma P$ is always invertible, so there is a unique solution 
$$
V_{\pi} = (I - \gamma P)^{-1} R.
$$

***
### Task 
Fill in the correct values of $R$ and $P$ in the code below, and see if you find the same solution as in the slides of Lecture 2. You can also play around with the discount rate, and/or try to compute $R$, $P$ and then $V_{\pi}$ for another policy.

**Note**: The state $Pass$ is a bit special, since it is a terminating state. Hence, when we reach "Pass the exam" we will not receive anymore rewards, and we will stay in this state forever (the probability of going to any other state is 0). Hence
$$
v_{\pi}(Pass) = 0 + \gamma \begin{bmatrix} 0 & 0 & 0 & 1 \end{bmatrix} V_{\pi}
$$
***

In [ ]:
discount = 0.9 # gamma
R = np.zeros((4,1))
P = np.zeros((4,4))

# Enter the expected immediate reward for each state
# Those computed in the text above are already filled in.
R[0] = -.5 
R[1] = -.5 # For K_1
R[2] = 5 
R[3] = 0 # For "Pass exam"

# Enter the probabilities going from state i to state j
P[0] = [0.5, 0.5, 0, 0] 
P[1] = [0.25, 0.25, 0.5, 0] # for i=1 (K_1)
P[2] = [0, 0.25, 0.25, 0.5] 
P[3] = [0, 0, 0, 1] # for i=3 (Pass the exam) 

# Solve the Bellman equation 
V = np.linalg.inv(np.eye(4) - discount*P)@R # V = (I - discount*P)^-1 * R
print(V)

# 3. Helper functions <a id="sec3">

We will now see how dynamic programming can be used to find the values and optimal actions in an MDP. 

First we will define two functions that will be used throughout the notebook. 

The class `RandomAgent` implements an agent with a policy $\pi(a|s)$. The method `act` will take the state $s$ as an argument, and then sample an action according to the probabilities $\pi(a|s)$. 

The probabilities are implemented in a table `probs` of size $|\mathcal{S}| \times |\mathcal{A}|$, so that `probs[s][a]` $= \pi(a|s)$. Here we implement an agent that (initially) chooses the action with a uniform probability, so all actions are equally likely in each state. However, by changing `probs` different policies can be implemented.

In [ ]:
class RandomAgent():
    
    def __init__(self, nA=4, nS=16):
        self.nA = nA # Number of actions
        self.nS = nS # Number of states
        
        # Uniform probabilites in each state.
        # That is, in each of the nS states
        # each of the nA actions has probability
        # 1/nA.
        self.probs = np.ones((nS,nA))/nA 

    def act(self, state):
        action = np.random.choice(self.nA, p=self.probs[state]) 
        return action # a random policy

We also define a function to run an environment with a given agent.

In [ ]:
def run_agent(env, agent):
    state, info = env.reset()
    time_step = 0
    total_reward = 0
    truncated = False
    terminated = False
    while not truncated and not terminated:
        action = agent.act(state);
        state, reward, terminated, truncated, info = env.step(action)
        total_reward += reward
        time_step += 1
        
        clear_output(wait=True)
        print("Time step:", time_step)
        print("State:", state)
        print("Action:", action)
        print("Total reward:", total_reward)
        
    if truncated:
        print("The environment was truncated even though a terminal state was not reached.")
    elif terminated:
        print("A terminal state was reached.")

# 4. The environments <a id="sec4">

In this notebook we will try the methods on different GridWorld environments. 
These environments are $4\times 4$ gridworlds. (It is possible to use larger grids, but in this notebook we will stay with $4 \times 4$ gridworlds)

<img src="grid.png" width=200>

The agent can be in one of the 16 positions, so the state space is
$$
\mathcal{S} = \{ 0, 1, 2, \ldots, 15\},
$$
see the figure for the meaning of each state. 

The action space contains four actions, 
$$
\mathcal{A} = \{ 0, 1, 2, 3\}
$$
corresponding to 

In [ ]:
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3

Let us look at Example 4.1 in the textbook. 

The player (here represented by a smiley) should go to one of the goals (which are supposed to look like ice creams with three cherries on top, but Per is not very good at drawing...)

___Note:___ If you prefer to use a different rendering mode, like `ansi`, also edit `run_agent` above by adding `print(env.render())` and potentially add `time.sleep(0.05)` at the end of the function.

In [ ]:
env = gym.make("GridWorld-v0", render_mode="human")
state, info = env.reset()
print("State space:", env.observation_space)
print("Action space:", env.action_space)

**The transition probabilities**: The environments used in this notebook have their transition probabilities $p(s', r | s, a)$ stored in `env.P`. Specifically, `env.P[s][a]` gives a list where each element is `(probability, next_state, reward, terminating)`. So, if you take action `a` in state `s`, then with probability `probability` you will move to `next_state` and get the reward `reward`. `terminating` tells us if `next_state` will terminate the episode or not. 

In [ ]:
s = 6
a = LEFT
print(env.P[s][a])
for p, next_s, reward, _ in env.P[s][a]: # Go through all possible transitions
    print("With probability", p, "you will move to state", next_s, 
          "and get the reward", reward)

Try a few sates and actions above and see that the results agree with your understanding of the environment. 

**Testing the environment:** To test the environment we create an agent that moves completely randomly. Then we use this agent on the environment. `run_agent` will continue until a terminating state (one of the "ice creams") is reached or the episode is truncated after 100 time steps. 

In [ ]:
agent = RandomAgent()
run_agent(env, agent)

**The optimal policy:** Note that `RandomAgent` choose actions according to the probabilities specified in `agent.probs`. By changing `agent.probs` we can thus change the policy of the agent. 

That is `agent.probs[s][a]` gives the probability of the agent choosing action `a` when it is in state `s`. 

**Task:** Implement the optimal policy for the environment. This policy is shown in Figure 4.1 in the textbook. Note there are several optimal actions in a given state, then you can choose between them with e.g. equal probability. 

In [ ]:
agent.probs = np.zeros((16, 4))

# Note that in each state the total
# probability must add upp to 1.0.

# You do not have to give any probabilities
# for state 0 or 15, since these
# are terminating states. 

# Row 1
agent.probs[1][LEFT] = 1.0
agent.probs[2][LEFT] = 1.0
agent.probs[3][[LEFT, DOWN]] = 0.5 # Pick between them with equal probability

# Row 2
agent.probs[4][UP] =  1.0 
agent.probs[5][[LEFT, UP]] = 0.5 
agent.probs[6][[DOWN, LEFT]] = 0.5 
agent.probs[7][DOWN] = 1.0 

# Row 3
agent.probs[8][UP] = 1.0 
agent.probs[9][[UP, RIGHT]] = 0.5 
agent.probs[10][[DOWN, RIGHT]] = 0.5 
agent.probs[11][DOWN] = 1.0 

# Row 4 
agent.probs[12][[UP, LEFT]] = 0.5 
agent.probs[13][RIGHT] = 1.0 
agent.probs[14][RIGHT] = 1.0 


In [ ]:
state, info = env.reset()
run_agent(env, agent)

Run the agent a few times to see that it seems to be acting optimally. 

## 4.1 The Frozen Lake <a id="sec4_1">

The environment `FrozenLake-v1` [(link)](https://gymnasium.farama.org/environments/toy_text/frozen_lake/) is similar to `GridWorld-v0`, but it is stochastic. 

In [ ]:
env = gym.make('FrozenLake-v1', render_mode="human")
state, info = env.reset()
print("State space:", env.observation_space)
print("Action space:", env.action_space)

The objective is to reach the goal without falling into a hole. However, the ice is slippery so the movement will only partially depend on your action. 

**Terminal states:** All holes and the goal are terminal states. If you fall into a hole you get no reward, but if you reach the goal you get a reward. 

**Reward:**
 All actions not leading to the goal state gives a reward of 0. If you get to the goal you get reward $+1$. Hence, to maximize the total reward the agent much reach the goal state without falling into a hole.
 

**Dynamics:** As mentioned above, this is a stochastic environment. As in the previous example, the transition probabilities are stored in `env.P`. 

In [ ]:
s = 0
a = RIGHT 
print(env.P[s][a])
for p, next_s, reward, _ in env.P[s][a]:
    print("With probability %.2f you will move to state %d and get reward %.1f." % (p, next_s, reward))

We see that the environment is stochastic in this case, since the action RIGHT in state 0 may take the agent either to state 4 (down), state 1 (right) or state 0 (stay), due to the slippery surface.

We can also try to run this environment using the random policy:

In [ ]:
agent = RandomAgent()
run_agent(env, agent)

We can see that typically the agent ends up in one of the holes, and thus the total reward is typically 0 (but the expected total reward is positive, since there is a non-zero probability that we reach the goal).

In [ ]:
env.close()

# 5. MDPs and the Bellman equations <a id="sec5">

In this section we will study the Bellman equations for the value function a bit closer. Remember that we defined the return as 
$$
G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \ldots
$$
and the state-value function as 
$$
v_\pi(s) = \mathbb{E}[ G_t | S_{t} = s].
$$
The Bellman equation for the state-value function is then 
$$
v_\pi(s) = \sum_{a} \pi(a|s) \sum_{r} \sum_{s'} p(s', r | s, a) [ r + \gamma v_\pi(s') ] = \sum_{a} \pi(a|s) q_{\pi}(s,a)
$$
where
$$
q_{\pi}(s,a) = \mathbb{E}_{\pi}[ G_t | S_t = s, A_t = a] = \sum_{r}\sum_{s'} p(s',r | s,a) [ r + \gamma v_{\pi}(s')]
$$
is the action-value function.

**Implementation:**
In the code we will represent the state-value function $v_\pi(s)$ as a vector $v$ with one element for each state in $\mathcal{S}$.

We will now implement functions for computing the right-hand side of the Bellman equation. 

**Task:**
Complete `compute_action_value` and `Bellman_RHS`. Make sure that you understand the code.

We start by a function that computes the action values $q_{\pi}(s,a)$ given the state-value function $v_\pi(s)$.

In [ ]:
def compute_action_value(env, discount, s, a, v):
    
    action_value = 0
    
    # Loop through all possible (s', r) pairs
    for p, next_s, reward, _ in env.P[s][a]:
        action_value += p * (reward + discount*v[next_s]) 
    
    return action_value

With the action values, we can now compute $\sum_{a} \pi(a|s) q_{\pi}(s,a)$ (the expected action value) to get the right-hand side (RHS) of the Bellman equation.

For this we use `agent.probs[s][a]` $=\pi(a|s)$, see discussion in "Helper Functions".

In [ ]:
def Bellman_RHS(env, discount, agent, s, v):
    
    state_value = 0
    
    for a in range(env.action_space.n):
        # Loop through all possible actions
        state_value += agent.probs[s][a]*compute_action_value(env, discount, s, a, v) 
    
    return state_value

Finally we implement a function that, given a value function, computes the right-hand side of the Bellman equation for all states.

In [ ]:
def Bellman_RHS_all(env, discount, agent, v0):
    # v0 is the given value function
    # v will be the right-hand side of the Bellman equation
    # If v0 is indeed the value function, then we should get v = v0.
    
    v = np.zeros(env.observation_space.n)
    
    for s in range(env.observation_space.n):
        v[s] = Bellman_RHS(env, discount, agent, s, v0)
    
    return v

## 5.1 Test your code on Example 4.1 (GridWorld) <a id="sec5_1">

For Example 4.1 we will consider the state-value $v_{\pi}(s)$ for the policy when each action is taken with equal probability. The discount rate is
$$
\gamma = 1
$$
The value function for this policy is given in Figure 4.1 in the textbook (the lower left), and it is

In [ ]:
v = np.array([[0, -14, -20, -22], 
             [-14, -18, -20, -20],
             [-20, -20, -18, -14], 
             [-22, -20, -14, 0]]).ravel()

print("v as vector:", v)
print("v as matrix:\n", v.reshape(4,4))

# ravel turns the matrix into an array,
# and with reshape we print it as a matrix again so that it is easier to read.

We can now use our code to see if this value function really satisfy the Bellman equation:

In [ ]:
env = gym.make('GridWorld-v0')
agent = RandomAgent()
discount = 1
v_new = Bellman_RHS_all(env, discount, agent, v)
print('Right-hand side of Bellman equation:\n', v_new.reshape(4,4))

**Task:** `v` is the true value-function for the policy $\pi$ implemented in `agent`. Hence, `v_new` (the right-hand side of the Bellman equation) should be equal to `v`. 

If `v_new` is not equal to `v`, go back and fix your code for `compute_action_value` and `Bellman_RHS`. Remember to re-run the code cells after you have changed the code!

# 6. Policy Evaluation <a id="sec6">

In Lecture 3 we learned that one way of solving the Bellman equation, is to start from an initial guess and then repeatedly update the value function by applying the right-hand side of the Bellman equation. 

Below is one way to implement this. The iteration will stop when the maximum change in $v$ is less than `tol` (tolerance) or the number of iterations are `max_iter`.

***Note:*** For this code to work properly, your implementation of `compute_action_value` and `Bellman_RHS` must be correct. So make sure that you have tested your code first!

**Task:** Make sure that you understand the code!

In [ ]:
def policy_evaluation(env, discount, agent, v0, max_iter=1000, tol=1e-6):
    
    v_old = v0
    
    for i in range(max_iter):
        v_new = Bellman_RHS_all(env, discount, agent, v_old)
        
        if np.max(np.abs(v_new-v_old)) < tol:
            break
            
        v_old = v_new
        
    return v_new

Lets try this on the `GridWorld-v0` example, with the uniformly random policy. We start with an initial guess $v_{\pi}(s) = 0$ for all $s$.

In [ ]:
env = gym.make('GridWorld-v0')
agent = RandomAgent()
discount = 1
v0 = np.zeros((env.observation_space.n))

v = policy_evaluation(env, discount, agent, v0)
print(v.reshape(4,4))

Is this (approximately) the same as the true value function in Figure 4.1 $(k=\infty)$?
If you do not find the correct value function, make sure that your code in `compute_action_value` and `Bellman_RHS` is correct!

To replicate the other parts of Figure 4.1, you can set `max_iter` in order to see how the value function looks after a few iterations.

In [ ]:
v0 = np.zeros((env.observation_space.n))
v = policy_evaluation(env, discount, agent, v0, max_iter=10)
print(v.reshape(4,4))

**Task:** Use `policy_evaluation` to compute the value function for `FrozenLake-v0` when the uniformly random policy is used. Use $\gamma = 1$.

In [ ]:
env = gym.make('FrozenLake-v1')
agent = RandomAgent()
discount = 1
# Write code for computing the state-value function
v0 = np.zeros(env.observation_space.n) 
v = policy_evaluation(env, discount, agent, v0) 
print(v.reshape(4,4))

## 6.1 In place updates <a id="sec6_1">

As mentioned in Lecture 3 and in the textbook, the policy evaluation is often implemented using in place updates.

This can both simplify implementation, since we do not keep two separate arrays, and it can also speed up convergence.

**Task:** Complete the code in `policy_evaluation_ip`. Pseudo-code can be found in the textbook in the box on page 75. Then test your code on `GridWorld-v0` with the uniform policy to see that you still get the correct value function.

***Note:*** You have already written a function that computes the right-hand side of the Bellman equation!

In [ ]:
def policy_evaluation_ip(env, discount, agent, v0, max_iter=1000, tol=1e-6):
    
    v = v0
    
    for i in range(max_iter): # Loop
        delta = 0
        for s in range(env.observation_space.n):
            vs = v[s]
            
            v[s] = Bellman_RHS(env, discount, agent, s, v) 
            
            delta = np.max([delta, np.abs(vs-v[s])])
            
        if (delta < tol): # Until delta < tol
            break
            
    return v    

In [ ]:
env = gym.make('GridWorld-v0')
agent = RandomAgent()
discount = 1

v0 = np.zeros(16)
v = policy_evaluation_ip(env, discount, agent, v0)
print(v.reshape(4,4))

# 7. Policy Iteration <a id="sec7">

Now when we have code for evaluating a policy, it is time to see how it can be improved. Remember that the idea is to act greedily with respect to $v_{\pi}(s)$. That is, given $v_{\pi}(s)$ we can compute $q_{\pi}(s,a)$, and then the greedy (improved) policy is
$$
\pi'(s) = \text{argmax}_{a} q_{\pi}(s,a)
$$
We have already written code for computing $q_{\pi}(s,a)$ for a given $v_{\pi}(s)$, so the only thing we have to do now is to implement the maximization.

`greedy_policy` will return `a_probs` which encode a policy that is greedy with respect to `v`. That is `a_probs[s][a]` $= \pi'(a|s)$. Make sure that you understand the code.

In [ ]:
def greedy_policy(env, discount, agent, v):
    
    # The new policy will be a_probs
    # We start by setting all probabilities to 0
    # Then when we have found the greedy action in a state, 
    # we change the probability for that action to 1.0.
    
    a_probs = np.zeros((env.observation_space.n, env.action_space.n)) 
    
    for s in range(env.observation_space.n):
        
        action_values = np.zeros(env.action_space.n)
        
        for a in range(env.action_space.n):
            # Compute action value for all actions
            action_values[a] = compute_action_value(env, discount, s, a, v)
            
        a_max = np.argmax(action_values) # A greedy action
        a_probs[s][a_max] = 1.0 # Always choose the greedy action!
        
    return a_probs

Lets try to improve the policy on `GridWorld-v0`.

In [ ]:
env = gym.make('GridWorld-v0', render_mode="human")
agent = RandomAgent()
discount = 1

# We first evaluate the policy
v = np.zeros(env.observation_space.n)
v = policy_evaluation(env, discount, agent, v)

In [ ]:
v_old = v

# And then we improve the policy (act greedy w.r.t v)
agent.probs = greedy_policy(env, discount, agent, v)

# We can also evaluate the new policy 
v = policy_evaluation(env, discount, agent, v)

print("Value of initial policy:")
print(v_old.reshape(4,4))
print("\nValue of improved policy:")
print(v.reshape(4,4))

Assuming that your implementation of `compute_action_value` is correct, 
we can clearly see that the improved policy has higher value in every state. In fact, the policy is now an optimal policy. To see this, you can try to rerun the second cell above and note that the policy does not improve anymore.

**Policy iteration:** However, it is not the case for all environments that the policy will converge to the optimal in just one improvement. Typically we have to improve the policy several times until it finally converge to the optimal policy. 

Finally, we can try to run the agent with the improved policy.

In [ ]:
run_agent(env, agent)

**Task:** Find an optimal policy for `FrozenLake-v1`. (This time it is not enough to do policy improvement only one time to reach an optimal policy. So do several improvements!)

In [ ]:
env = gym.make('FrozenLake-v1', render_mode='human')
agent = RandomAgent()
discount = 1
# Enter code here
v_old = np.zeros(env.observation_space.n) 
for i in range(1000): 
    v = policy_evaluation(env, discount, agent, v_old) 
    
    if (np.max(np.abs(v-v_old))<1e-6): 
        break 
        
    v_old = v 
    agent.probs = greedy_policy(env, discount, agent, v) 

print(v.reshape(4,4)) 


When you have found a new policy you can try it out on the environment. 

___Note:___ You can see from above that the value of each state is strictly less than one even for the optimal policy. Since you get a total reward of $+1$ if you reach the goal, and total reward 0 if you fall into a hole, this means that even for the optimal policy there is a non-zero probability that you will fall into a hole. However, the probability that you reach the goal is much higher for the optimal policy than it was for the uniformly random policy. 

In [ ]:
run_agent(env, agent)

# 8. Value iteration <a id="sec8">

In the value iteration we instead start from the Bellman optimality equation

$$
v_{*}(s) = \max_{a} q_{\pi_*}(s,a) = \max_a \sum_{s', r} p(s', r | s, a) [r + \gamma v_{*}(s')]
$$

We start with an initial guess $v_0$ and then we repeatedly compute the right-hand side of this equation, until we converge to the optimal state-value function. When we have the optimal state-value function $v_*$, we can take any policy that is greedy w.r.t $v_*$ and this will give us an optimal policy. 

**Task 1:** Complete the code below. Pseudo-code for the algorithm can be found on page 83 in the textbook. Note that the code for computing the action-value given $v_{\pi}$ has already been implemented above.

The `value_iteration` function will (if implemented correctly) give back the optimal value function. 

**Task 2:** Also add some code for computing the optimal policy given this, and try it on `FrozenLake-v0` and/or `GridWorld-v0`.

In [ ]:
def value_iteration(env, discount, agent, v0, max_iter=1000, tol=1e-6):
    
    v = v0
    
    for i in range(max_iter): # Loop
        delta = 0
        for s in range(env.observation_space.n):
            vs = v[s]
            
            action_values = np.zeros(env.action_space.n) 
            
            for a in range(env.action_space.n): 
                action_values[a] = compute_action_value(env, discount, s, a, v) 
            
            v[s] = np.max(action_values) 
            
            delta = np.max([delta, np.abs(vs-v[s])])
            
        if (delta < tol): # Until delta < tol
            break
            
    print(i) 
            
    return v    

In [ ]:
env = gym.make('FrozenLake-v1', render_mode="human")
agent = RandomAgent()
discount = 1

v0 = np.zeros(env.observation_space.n) 
v = value_iteration(env, discount, agent, v0) 
agent.probs = greedy_policy(env, discount, agent, v) 
print(v.reshape(4,4)) 


**Checking your results:** If everything is done correctly, the value iteration should find the same optimal value function as before. 